## **1) Importing Python Packages for GAN**
Kode ini mempersiapkan dataset MNIST, mengimpor berbagai lapisan dan optimizer dari Keras untuk membangun model deep learning, dan membuat folder untuk menyimpan gambar yang dihasilkan.

In [2]:
from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dense, Reshape, Flatten
from keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam

import numpy as np
!mkdir generated_images

mkdir: cannot create directory ‘generated_images’: File exists


## **2) Variables for Neural Networks & Data**
mendefinisikan parameter untuk model generatif, seperti dimensi gambar (28x28 piksel, grayscale), ruang laten berukuran 100, dan menggunakan optimizer Adam dengan laju pembelajaran 0.0001.

In [3]:
img_width = 28
img_height = 28
channels = 1
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(learning_rate=0.0001)

## **3) Building Generator**
Fungsi ini membangun model generator untuk menghasilkan gambar. Model ini menggunakan beberapa lapisan Dense dengan aktivasi LeakyReLU dan BatchNormalization untuk meningkatkan stabilitas pelatihan. Di akhir, gambar dihasilkan dengan aktivasi tanh dan diubah bentuknya menjadi ukuran gambar yang diinginkan (28x28x1). Model generator ini diinisialisasi dengan dimensi ruang laten dan menghasilkan gambar dengan bentuk yang sesuai.


In [4]:
def build_generator():
  model = Sequential()

  model.add(Dense(256, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))

  model.summary()
  return model

generator = build_generator()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 256)                 │          25,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 784)                 │         201,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 28, 28, 1)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 362,000 (1.38 MB)

 Trainable params: 360,464 (1.38 MB)

 Non-trainable params: 1,536 (6.00 KB)

## **4) Building Discriminator**

Fungsi ini membangun model diskriminator untuk membedakan gambar asli dan gambar yang dihasilkan oleh generator. Model ini memulai dengan lapisan Flatten untuk mengubah gambar menjadi vektor, diikuti oleh lapisan Dense dengan aktivasi LeakyReLU, dan akhirnya lapisan output dengan aktivasi sigmoid untuk menghasilkan prediksi antara 0 (gambar palsu) dan 1 (gambar asli). Diskriminator ini kemudian dikompilasi menggunakan loss binary_crossentropy dan optimizer Adam.

In [5]:
def build_discriminator():
  model = Sequential()

  model.add(Flatten(input_shape=img_shape))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(256))
  model.add(Dense(1, activation='sigmoid'))

  model.summary()
  return model

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │         401,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 533,505 (2.04 MB)

 Trainable params: 533,505 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

## **5) Connecting Neural Networks to build GAN**
Membangun model GAN (Generative Adversarial Network) dengan menggabungkan generator dan diskriminator. Diskriminator diatur agar tidak dapat dilatih (discriminator.trainable = False) saat pelatihan GAN, sehingga hanya generator yang diperbarui selama pelatihan. Model GAN ini kemudian dikompilasi dengan loss binary_crossentropy dan optimizer Adam. Setelah itu, ringkasan model ditampilkan.

In [6]:
GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)

GAN.compile(loss='binary_crossentropy', optimizer=adam)
GAN.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ (None, 28, 28, 1)           │         362,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_1 (Sequential)            │ (None, 1)                   │         533,505 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 895,505 (3.42 MB)

 Trainable params: 360,464 (1.38 MB)

 Non-trainable params: 535,041 (2.04 MB)

## **6) Outputting Images**
Menghasilkan gambar dari generator dan menyimpannya sebagai file PNG. Setiap kali dipanggil, fungsi membuat 25 gambar (5x5 grid) dengan menghasilkan noise acak, mengubahnya menjadi gambar oleh generator, dan menyesuaikan skala gambar ke rentang 0-1. Gambar yang dihasilkan kemudian disimpan dengan nama file yang meningkat secara bertahap.

In [7]:
#@title
## **7) Outputting Images**
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001
    print("%.8f" % save_name)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            # axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("generated_images/%.8f.png" % save_name)
    print('saved')
    plt.close()

## **7) Training GAN**
Melatih model GAN untuk menghasilkan gambar. Dalam setiap epoch, fungsi ini:

1. Mengambil batch acak dari dataset MNIST dan mengubah skala gambar ke rentang -1 hingga 1.
2. Menghasilkan gambar palsu menggunakan generator.
3. Melatih diskriminator dengan gambar asli dan palsu untuk menghitung kerugian (d_loss).
4. Menghasilkan noise acak dan melatih generator melalui GAN untuk meminimalkan kerugian (g_loss).
5. Setiap interval tertentu (misalnya setiap 200 epoch), gambar yang dihasilkan disimpan.

Fungsi ini menjalankan pelatihan selama sejumlah epoch yang ditentukan dan mencetak kerugian serta akurasi diskriminator dan generator di setiap langkah.


In [8]:
def train(epochs, batch_size=64, save_interval=200):
  (X_train, _), (_, _) = mnist.load_data()

  # print(X_train.shape)
  #Rescale data between -1 and 1
  X_train = X_train / 127.5 -1.
  # X_train = np.expand_dims(X_train, axis=3)
  # print(X_train.shape)

  #Create our Y for our Neural Networks
  valid = np.ones((batch_size, 1))
  fakes = np.zeros((batch_size, 1))

  for epoch in range(epochs):
    #Get Random Batch
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]

    #Generate Fake Images
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_imgs = generator.predict(noise)

    #Train discriminator
    d_loss_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    noise = np.random.normal(0, 1, (batch_size, latent_dim))

    #inverse y label
    g_loss = GAN.train_on_batch(noise, valid)

    # Access the first element of g_loss, which represents the total loss
    g_loss_value = g_loss[0]

    print("******* %d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100* d_loss[1], g_loss_value))

    if(epoch % save_interval) == 0:
      save_imgs(epoch)

  # print(valid)


train(500, batch_size=64, save_interval=200)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


******* 0 [D loss: 0.289592, acc: 87.50%] [G loss: 0.423206]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
0.00000001
saved
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


******* 1 [D loss: 0.376804, acc: 78.19%] [G loss: 0.424197]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
******* 2 [D loss: 0.404660, acc: 74.11%] [G loss: 0.438588]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
******* 3 [D loss: 0.421767, acc: 71.33%] [G loss: 0.444166]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
******* 4 [D loss: 0.434852, acc: 69.07%] [G loss: 0.458123]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
******* 5 [D loss: 0.446164, acc: 67.34%] [G loss: 0.462725]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
******* 6 [D loss: 0.455207, acc: 66.33%] [G loss: 0.470917]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
******* 7 [D loss: 0.465263, acc: 65.08%] [G loss: 0.480230]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
******* 8 [D loss: 0.473257, acc: 64.23%] [G loss: 0.485234]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
******* 9 [D loss: 0.479602, acc: 63.54%] [G loss: 0.491264]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
******* 10 [D loss: 0.487108, acc: 62.64%] [G loss: 0.498942]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
******* 1

### **8) Making GIF**
Membuat animasi GIF dari gambar yang dihasilkan selama pelatihan. Dengan menggunakan imageio, gambar yang disimpan dalam folder generated_images dibaca dan diurutkan. Gambar-gambar tersebut kemudian digabungkan menjadi sebuah file GIF (dcgan.gif) yang menampilkan proses pembuatan gambar dari generator selama pelatihan.

In [9]:
# Display a single image using the epoch number
# def display_image(epoch_no):
#   return PIL.Image.open('generated_images/%.8f.png'.format(epoch_no))

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

<ipython-input-9-5d911b6b7554>:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
<ipython-input-9-5d911b6b7554>:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
